In [1]:
import re
import cv2
import pandas as pd
import os
from datetime import datetime
# from pyzbar.pyzbar import decode

In [2]:
# -------- Extract UPI ID from link ----------
def extract_upi_id_from_link(link):
    match = re.search(r'pa=([^&]+)', link)
    return match.group(1) if match else None

# -------- Read QR Code and extract UPI ID ----------
def extract_qr_data(image_path):
    detector = cv2.QRCodeDetector()
    img = cv2.imread(image_path)
    data, bbox, _ = detector.detectAndDecode(img)
    return data

# -------- Unified input handler ----------
def get_upi_input():
    print("Select Input Type:")
    print("1. Enter UPI ID manually")
    print("2. Enter UPI payment link")
    print("3. Upload QR code image")

    choice = input("Enter choice (1/2/3): ").strip()

    if choice == '1':
        upi_id = input("Enter UPI ID (e.g., abc@upi): ").strip()
    elif choice == '2':
        link = input("Paste UPI link (e.g., upi://pay?...): ").strip()
        upi_id = extract_upi_id_from_link(link)
        if not upi_id:
            print("Could not extract UPI ID from link.")
            return None
    elif choice == '3':
        path = input("Enter image path (e.g., ./qr.png): ").strip()
        upi_id = extract_qr_data(path)
        if not upi_id:
            print("No valid QR code found.")
            return None
    else:
        print("Invalid choice.")
        return None

    print(f"\n✅ Extracted UPI ID: {upi_id}")
    return upi_id


In [3]:
def community_report(upi_id, message, reason, reporter=None):
    """
    Stores community-reported scam data for future scoring/tracking.
    """
    report = {
        "upi_id": upi_id,
        "message": message,
        "reason": reason,
        "reporter": reporter if reporter else "anonymous",
        "timestamp": datetime.now().isoformat()
    }

    file_path = "community_reports.csv"

    # Append or create
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        df = pd.concat([df, pd.DataFrame([report])], ignore_index=True)
    else:
        df = pd.DataFrame([report])

    df.to_csv(file_path, index=False)
    print("✅ Report saved. Thank you for contributing!")

    return True
